In [1]:
import numpy as np 
import pandas as pd 
import re 
import os 
import glob
from pathlib import Path
import collections
import numexpr
from enum import Enum, auto
from multiprocessing import cpu_count
from collections import UserString
#import ray
#ray.init()
numexpr.set_num_threads(cpu_count())

4

In [2]:
ROOT_PATH=os.path.abspath(os.curdir)
PATH_DIR_DATA="data/kara"
ALL_DIR_ON_DATA=glob.glob(f'{os.path.join(ROOT_PATH,PATH_DIR_DATA)}/*/')

ROOT_DIR_RAW=list(filter(re.compile(".*raw").match, ALL_DIR_ON_DATA))[0]
ROOT_DIR_BURNET=list(filter(re.compile(".*burnet").match, ALL_DIR_ON_DATA))[0]
ROOT_DIR_ZIPPED=list(filter(re.compile(".*zipped").match, ALL_DIR_ON_DATA))[0]

ALLXML_FILES = list(Path(ROOT_DIR_RAW).glob('**/*.xml'))

In [3]:
ALLXML_FILES[0]
ALLXML_FILES[0].name

'PPM-R15m-NE13143-2018_09_03-19_30.xml'

In [4]:
def add_constants(names:dict):
    def adder(cls):
        for key, value in names.items():
            setattr(cls, key, value)
        return cls
    return adder

class AutoName(Enum):
    def _generate_next_value_(name, start, count, last_values):
        return name
    
class AutoValue(Enum):
    def _generate_next_value_(name, start, count, last_values):
        return count
    
class Columns(AutoValue): 
    EntityId=auto()
    neId=auto()
    neAlias=auto()
    neType=auto()
    Time=auto()
    Failure=auto()
    EntityType=auto()
    DropEvents=auto()
    TotlOctets=auto()
    TotPackets=auto()
    BroadcPack=auto()
    MulticPack=auto()
    CrcAlgnErr=auto()
    UndSizPack=auto()
    OvrSizPack=auto()
    Fragments=auto()
    Jabbers=auto()
    Collisions=auto()
    Utilizatio=auto()
    ElementID=auto()
    
@add_constants({"NE":".//NE","NEName":".//NE/NEIdentity/NEName","NEType":".//NE/NEIdentity/NEType","Entity":".//Entity","Compliance":".//Entity/Details/Compliance","TimeStamp":".//Entity/Details/TimeStamp","EntityIdentity":".//Entity/EntityIdentity","Counter":".//Entity/Details/Counter"})               
class Elements:
    pass
    
    

    

In [5]:
NEName = pd.read_xml(ALLXML_FILES[0],
                        xpath=Elements.NEName,attrs_only=True)

NEType = pd.read_xml(ALLXML_FILES[0],
                        xpath=Elements.NEType)

Entity =pd.read_xml(ALLXML_FILES[0],
                        xpath=Elements.Entity,attrs_only=True)

Compliance = pd.read_xml(ALLXML_FILES[0],
                        xpath=Elements.Compliance)

TimeStamp = pd.read_xml(ALLXML_FILES[0],
                        xpath=Elements.TimeStamp,attrs_only=True)

Counter = pd.read_xml(ALLXML_FILES[0],
                        xpath=Elements.Counter)

EntityIdentity=pd.read_xml(ALLXML_FILES[0],
                        xpath=Elements.EntityIdentity)

In [6]:
list(Columns)

[<Columns.EntityId: 0>,
 <Columns.neId: 1>,
 <Columns.neAlias: 2>,
 <Columns.neType: 3>,
 <Columns.Time: 4>,
 <Columns.Failure: 5>,
 <Columns.EntityType: 6>,
 <Columns.DropEvents: 7>,
 <Columns.TotlOctets: 8>,
 <Columns.TotPackets: 9>,
 <Columns.BroadcPack: 10>,
 <Columns.MulticPack: 11>,
 <Columns.CrcAlgnErr: 12>,
 <Columns.UndSizPack: 13>,
 <Columns.OvrSizPack: 14>,
 <Columns.Fragments: 15>,
 <Columns.Jabbers: 16>,
 <Columns.Collisions: 17>,
 <Columns.Utilizatio: 18>,
 <Columns.ElementID: 19>]

In [7]:
Entity=Entity.drop(columns=["sourceId","shelfId","equipId","emId"])
%time Entity[Columns.neId.name]=int(Entity.neId.values[0].split("-")[-1])
#%time Entity['neId'].map(lambda row: row.neId.split("-")[-1])
Entity= Entity.rename(columns={"Id":Columns.EntityId.name})
Entity

CPU times: user 515 µs, sys: 23 µs, total: 538 µs
Wall time: 550 µs


,EntityId,neId
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143


In [8]:
re.findall(r'[A-Z][A-Z][0-9]*[0-9][^/]', NEName.longName.values[0])[0]

'MA0950'

In [9]:
NEName_val=re.findall(r'[A-Z][A-Z][0-9]*[0-9][^/]', NEName.longName.values[0])[0]
NEName_val
Entity.insert(2, Columns.neAlias.name, NEName_val)
Entity

,EntityId,neId,neAlias
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950


In [10]:
NEType_val=NEType.NEType.values[0]
NEType_val=NEType_val.replace(re.findall(r'[A-Z]*[A-Z][^0-9]', 
                              NEType_val)[0],f"{re.findall(r'[A-Z]*[A-Z][^0-9]', NEType_val)[0]} ")
NEType_val

'SPO 1410'

In [11]:
Entity.insert(3, Columns.neType.name, NEType_val)
Entity

,EntityId,neId,neAlias,neType
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410


In [12]:
%time Entity=pd.concat([Entity,TimeStamp.rename(columns={"localTimeFormatId":"Time"})],axis=1)
Entity

CPU times: user 1.21 ms, sys: 56 µs, total: 1.26 ms
Wall time: 1.26 ms


,EntityId,neId,neAlias,neType,Time
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00


In [13]:
Compliance = Compliance.drop(columns=["complianceId","complianceType"]).rename(columns={"Compliance":Columns.Failure.name})
%time Entity=pd.concat([Entity,Compliance],axis=1)
Entity

CPU times: user 920 µs, sys: 56 µs, total: 976 µs
Wall time: 987 µs


,EntityId,neId,neAlias,neType,Time,Failure
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success


In [14]:
EntityIdentity = EntityIdentity.drop(columns=["entityTypeId"]).rename(columns={"EntityIdentity":Columns.EntityType.name})
%time Entity=pd.concat([Entity,EntityIdentity],axis=1)
Entity

CPU times: user 986 µs, sys: 25 µs, total: 1.01 ms
Wall time: 1.01 ms


,EntityId,neId,neAlias,neType,Time,Failure,EntityType
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON


In [15]:
Counter

,name,Counter
0,DropEvents,0.000000e+00
1,TotlOctets,2.083125e+11
2,TotPackets,6.359410e+08
3,BroadcPack,5.333000e+03
4,MulticPack,1.340900e+04
...,...,...
199,OvrSizPack,0.000000e+00
200,Fragments,0.000000e+00
201,Jabbers,0.000000e+00
202,Collisions,0.000000e+00


In [16]:
#%time Counter.name.apply(pd.value_counts).to_dict().keys()
%time count_names = list(collections.Counter(Counter.name.to_numpy()))
count_names

CPU times: user 374 µs, sys: 46 µs, total: 420 µs
Wall time: 434 µs


['DropEvents',
 'TotlOctets',
 'TotPackets',
 'BroadcPack',
 'MulticPack',
 'CrcAlgnErr',
 'UndSizPack',
 'OvrSizPack',
 'Fragments',
 'Jabbers',
 'Collisions',
 'Utilizatio']

In [17]:
%time list_filter_Counter = list(map(lambda name: Counter.query(f'name == "{name}"',engine="numexpr"), count_names))

CPU times: user 33.4 ms, sys: 1.75 ms, total: 35.2 ms
Wall time: 35.9 ms


In [18]:
list_filter_Counter[0]

,name,Counter
0,DropEvents,0.0
12,DropEvents,0.0
24,DropEvents,0.0
36,DropEvents,0.0
48,DropEvents,0.0
60,DropEvents,0.0
72,DropEvents,0.0
84,DropEvents,0.0
96,DropEvents,0.0
108,DropEvents,0.0


In [19]:
%time result_fiter= list(map(lambda name,df :df.rename({"Counter":name},axis=1).drop( ["name"], axis=1).reset_index().drop(['index'],axis=1) ,count_names,list_filter_Counter))
Counter=pd.concat(result_fiter,axis=1)

CPU times: user 35.7 ms, sys: 1.52 ms, total: 37.2 ms
Wall time: 38.2 ms


In [20]:
Counter

,DropEvents,TotlOctets,TotPackets,BroadcPack,MulticPack,CrcAlgnErr,UndSizPack,OvrSizPack,Fragments,Jabbers,Collisions,Utilizatio
0,0.0,2.083125e+11,635941045.0,5333.0,13409.0,0.0,0.0,0.0,0.0,0.0,0.0,19.6
1,0.0,9.620172e+09,71634941.0,3.0,990.0,0.0,0.0,134804.0,0.0,0.0,0.0,9.8
2,0.0,3.939395e+08,2539525.0,1.0,0.0,0.0,0.0,10044.0,0.0,0.0,0.0,0.3
3,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,8.028546e+09,60882891.0,12.0,990.0,0.0,0.0,156283.0,0.0,0.0,0.0,8.2
5,0.0,1.752692e+09,12597619.0,1.0,0.0,0.0,0.0,32805.0,0.0,0.0,0.0,1.7
6,0.0,8.293324e+09,62675626.0,4.0,994.0,0.0,0.0,65367.0,0.0,0.0,0.0,8.4
7,0.0,1.371270e+09,11957380.0,1.0,0.0,0.0,0.0,380.0,0.0,0.0,0.0,1.4
8,0.0,1.387770e+09,9991279.0,2.0,0.0,0.0,0.0,4851.0,0.0,0.0,0.0,1.4
9,0.0,1.131743e+09,9486460.0,0.0,991.0,0.0,0.0,13417.0,0.0,0.0,0.0,1.1


In [21]:
%time Entity=pd.concat([Entity,Counter],axis=1)
Entity

CPU times: user 946 µs, sys: 120 µs, total: 1.07 ms
Wall time: 987 µs


,EntityId,neId,neAlias,neType,Time,Failure,EntityType,DropEvents,TotlOctets,TotPackets,BroadcPack,MulticPack,CrcAlgnErr,UndSizPack,OvrSizPack,Fragments,Jabbers,Collisions,Utilizatio
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,2.083125e+11,635941045.0,5333.0,13409.0,0.0,0.0,0.0,0.0,0.0,0.0,19.6
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,9.620172e+09,71634941.0,3.0,990.0,0.0,0.0,134804.0,0.0,0.0,0.0,9.8
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,3.939395e+08,2539525.0,1.0,0.0,0.0,0.0,10044.0,0.0,0.0,0.0,0.3
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,8.028546e+09,60882891.0,12.0,990.0,0.0,0.0,156283.0,0.0,0.0,0.0,8.2
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.752692e+09,12597619.0,1.0,0.0,0.0,0.0,32805.0,0.0,0.0,0.0,1.7
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,8.293324e+09,62675626.0,4.0,994.0,0.0,0.0,65367.0,0.0,0.0,0.0,8.4
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.371270e+09,11957380.0,1.0,0.0,0.0,0.0,380.0,0.0,0.0,0.0,1.4
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.387770e+09,9991279.0,2.0,0.0,0.0,0.0,4851.0,0.0,0.0,0.0,1.4
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.131743e+09,9486460.0,0.0,991.0,0.0,0.0,13417.0,0.0,0.0,0.0,1.1


In [22]:
%time ElementID = Entity.apply(lambda raw : f"{raw.neType}_{raw.EntityType}({raw.EntityId})".replace(" ","_") ,axis=1)
%time Entity=pd.concat([Entity,ElementID],axis=1)
%time Entity.columns = [*Entity.columns[:-1], Columns.ElementID.name]
Entity

CPU times: user 2.09 ms, sys: 46 µs, total: 2.13 ms
Wall time: 2.15 ms
CPU times: user 1.64 ms, sys: 51 µs, total: 1.69 ms
Wall time: 1.8 ms
CPU times: user 348 µs, sys: 1 µs, total: 349 µs
Wall time: 358 µs


,EntityId,neId,neAlias,neType,Time,Failure,EntityType,DropEvents,TotlOctets,TotPackets,BroadcPack,MulticPack,CrcAlgnErr,UndSizPack,OvrSizPack,Fragments,Jabbers,Collisions,Utilizatio,ElementID
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,2.083125e+11,635941045.0,5333.0,13409.0,0.0,0.0,0.0,0.0,0.0,0.0,19.6,SPO_1410_RMON(EN-0-13143-0-0-3-9-1-0-0-0-0-21-0)
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,9.620172e+09,71634941.0,3.0,990.0,0.0,0.0,134804.0,0.0,0.0,0.0,9.8,SPO_1410_RMON(EN-0-13143-0-0-4-9-2-0-0-0-0-21-0)
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,3.939395e+08,2539525.0,1.0,0.0,0.0,0.0,10044.0,0.0,0.0,0.0,0.3,SPO_1410_RMON(EN-0-13143-0-0-4-9-10-0-0-0-0-21-0)
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SPO_1410_RMON(EN-0-13143-0-0-4-9-1-0-0-0-0-21-0)
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,8.028546e+09,60882891.0,12.0,990.0,0.0,0.0,156283.0,0.0,0.0,0.0,8.2,SPO_1410_RMON(EN-0-13143-0-0-4-9-4-0-0-0-0-21-0)
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.752692e+09,12597619.0,1.0,0.0,0.0,0.0,32805.0,0.0,0.0,0.0,1.7,SPO_1410_RMON(EN-0-13143-0-0-4-9-5-0-0-0-0-21-0)
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,8.293324e+09,62675626.0,4.0,994.0,0.0,0.0,65367.0,0.0,0.0,0.0,8.4,SPO_1410_RMON(EN-0-13143-0-0-4-9-3-0-0-0-0-21-0)
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.371270e+09,11957380.0,1.0,0.0,0.0,0.0,380.0,0.0,0.0,0.0,1.4,SPO_1410_RMON(EN-0-13143-0-0-4-9-6-0-0-0-0-21-0)
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.387770e+09,9991279.0,2.0,0.0,0.0,0.0,4851.0,0.0,0.0,0.0,1.4,SPO_1410_RMON(EN-0-13143-0-0-4-9-7-0-0-0-0-21-0)
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.131743e+09,9486460.0,0.0,991.0,0.0,0.0,13417.0,0.0,0.0,0.0,1.1,SPO_1410_RMON(EN-0-13143-0-0-4-9-9-0-0-0-0-21-0)


In [23]:
Entity.columns

Index(['EntityId', 'neId', 'neAlias', 'neType', 'Time', 'Failure',
       'EntityType', 'DropEvents', 'TotlOctets', 'TotPackets', 'BroadcPack',
       'MulticPack', 'CrcAlgnErr', 'UndSizPack', 'OvrSizPack', 'Fragments',
       'Jabbers', 'Collisions', 'Utilizatio', 'ElementID'],
      dtype='object')

In [24]:
%time Entity.to_csv(ROOT_DIR_ZIPPED+ALLXML_FILES[0].name.replace("xml","csv"),index=False)

CPU times: user 4.45 ms, sys: 3.95 ms, total: 8.39 ms
Wall time: 8.46 ms


In [25]:
Entity.dtypes

EntityId       object
neId            int64
neAlias        object
neType         object
Time           object
Failure        object
EntityType     object
DropEvents    float64
TotlOctets    float64
TotPackets    float64
BroadcPack    float64
MulticPack    float64
CrcAlgnErr    float64
UndSizPack    float64
OvrSizPack    float64
Fragments     float64
Jabbers       float64
Collisions    float64
Utilizatio    float64
ElementID      object
dtype: object

In [28]:
import clickhouse_connect
client = clickhouse_connect.get_client(host='localhost', port=8123, username='default',)

In [29]:
CLICKHOUSE_TYPES={"object":"String","float64":"Float64","int64":"UInt64"}

In [36]:
ENGINE="MergeTree"
DB_NAME="default"
TABLE_NAME="_".join(ALLXML_FILES[0].name.split("-")[:3])
QUARY_CREATE=f'CREATE TABLE {TABLE_NAME} ({"".join([ f" {col} {CLICKHOUSE_TYPES[_type.name]}," for col , _type in Entity.dtypes.to_dict().items()])}) ENGINE {ENGINE} ORDER BY {Columns.Time.name}'.replace(",)",")")
QUARY_CREATE

'CREATE TABLE PPM_R15m_NE13143 ( EntityId String, neId UInt64, neAlias String, neType String, Time String, Failure String, EntityType String, DropEvents Float64, TotlOctets Float64, TotPackets Float64, BroadcPack Float64, MulticPack Float64, CrcAlgnErr Float64, UndSizPack Float64, OvrSizPack Float64, Fragments Float64, Jabbers Float64, Collisions Float64, Utilizatio Float64, ElementID String) ENGINE MergeTree ORDER BY Time'

In [32]:
client.command(QUARY_CREATE)


In [38]:
QUARY_CREATE=f"SHOW TABLES FROM {DB_NAME}"
client.command(QUARY_CREATE)

'PPM_R15m_NE13143'

In [39]:
QUARY_CREATE=f'DROP TABLE IF EXISTS {QUARY_CREATE}'
client.command(QUARY_CREATE)
client.insert_df(TABLE_NAME,Entity)

In [40]:
result_df = client.query_df(f'SELECT * FROM {TABLE_NAME}')
 client.close()
result_df

,EntityId,neId,neAlias,neType,Time,Failure,EntityType,DropEvents,TotlOctets,TotPackets,BroadcPack,MulticPack,CrcAlgnErr,UndSizPack,OvrSizPack,Fragments,Jabbers,Collisions,Utilizatio,ElementID
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,2.083125e+11,635941045.0,5333.0,13409.0,0.0,0.0,0.0,0.0,0.0,0.0,19.6,SPO_1410_RMON(EN-0-13143-0-0-3-9-1-0-0-0-0-21-0)
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,9.620172e+09,71634941.0,3.0,990.0,0.0,0.0,134804.0,0.0,0.0,0.0,9.8,SPO_1410_RMON(EN-0-13143-0-0-4-9-2-0-0-0-0-21-0)
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,3.939395e+08,2539525.0,1.0,0.0,0.0,0.0,10044.0,0.0,0.0,0.0,0.3,SPO_1410_RMON(EN-0-13143-0-0-4-9-10-0-0-0-0-21-0)
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SPO_1410_RMON(EN-0-13143-0-0-4-9-1-0-0-0-0-21-0)
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,8.028546e+09,60882891.0,12.0,990.0,0.0,0.0,156283.0,0.0,0.0,0.0,8.2,SPO_1410_RMON(EN-0-13143-0-0-4-9-4-0-0-0-0-21-0)
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.752692e+09,12597619.0,1.0,0.0,0.0,0.0,32805.0,0.0,0.0,0.0,1.7,SPO_1410_RMON(EN-0-13143-0-0-4-9-5-0-0-0-0-21-0)
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,8.293324e+09,62675626.0,4.0,994.0,0.0,0.0,65367.0,0.0,0.0,0.0,8.4,SPO_1410_RMON(EN-0-13143-0-0-4-9-3-0-0-0-0-21-0)
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.371270e+09,11957380.0,1.0,0.0,0.0,0.0,380.0,0.0,0.0,0.0,1.4,SPO_1410_RMON(EN-0-13143-0-0-4-9-6-0-0-0-0-21-0)
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.387770e+09,9991279.0,2.0,0.0,0.0,0.0,4851.0,0.0,0.0,0.0,1.4,SPO_1410_RMON(EN-0-13143-0-0-4-9-7-0-0-0-0-21-0)
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.131743e+09,9486460.0,0.0,991.0,0.0,0.0,13417.0,0.0,0.0,0.0,1.1,SPO_1410_RMON(EN-0-13143-0-0-4-9-9-0-0-0-0-21-0)


In [ ]:
 print(result_df.dtypes)

EntityId       object
neId            int64
neAlias        object
neType         object
Time           object
Failure        object
EntityType     object
DropEvents    float64
TotlOctets    float64
TotPackets    float64
BroadcPack    float64
MulticPack    float64
CrcAlgnErr    float64
UndSizPack    float64
OvrSizPack    float64
Fragments     float64
Jabbers       float64
Collisions    float64
Utilizatio    float64
ElementID      object
dtype: object
